<a href="https://colab.research.google.com/github/furtman/WQTC25_workshop_IMTR/blob/main/Ex_geospatial/water_quality_visualization_Python_ANSWERS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PCW01: From Data Management to Data Analysis — A Technical Deep-Dive**
AWWA WQTC 2025, November 9-13th, Tacoma WA
## **Water Quality Data Visualization using Google Colab and Python**

---

This notebook walks through building and running an interactive data visualization dashboard inside a **Google Colab** workspace using the **Python** programming language. Users will learn how to load, inspect, and analyze data from comma-separated (.csv) files, generate static and interactive visualizations of time-series plots and maps, and compile visualizations and stats into a PDF report and an easy-to-use dashboard that synthesizes data into single webpage.

The water quality data used in this notebook can be downloaded directly from the Thurston County Streams Water Quality Dashboard: https://www.thurstoncountywa.gov/streams-water-quality-dashboard

This link will take you to an example of a fully-functional, web-based data dashboard that was created using R (another popular programming langauge used for data science).


### 🌎 Notebook Table of Contents:
1.   Setup Python Runtime
2.   Load Python Libraries
3.   Upload Data Files to Google Colab
4.   Load and Inspect Data
5.   Data Cleaning and Processing
6.   Generating Summary Statistics
7.   Generating Plots
8.   Automating Plot Generation and PDF Reports
9.   Generating Maps
10.  Generating Data Dashboards

### 💪 Helpful keyboard shortcuts:
*   `ctrl enter`: Runs a code cell
*   `ctrl shift enter`: Runs a code cell and advances to next cell
*   `ctrl shift enter w/ selection`: Runs the selected code within a code cell








---
## 1. Setup Python Runtime
Prior to installing any libraries/packages, change the **runtime type** to **Python 3** if it isn't already selected. Google Colab comes with several popular python packages pre-installed, such as *pandas* and *matplotlib*.



---
## 2. Load Python Libraries
In order to use the functions within each library, we need to load them into the notebook.

In [ ]:
# Load in the Python libraries that will be used in this notebook
# Tip: Hover over the libraries to view more info about them

from google.colab import files                       # for uploading/downloading files in Colab
import pandas as pd                                  # data manipulation and analysis
import matplotlib.pyplot as plt                      # plotting static graphs
from scipy.stats import linregress                   # linear regression and statistical functions
import plotly.express as px                          # interactive plots and dashboards
from matplotlib.backends.backend_pdf import PdfPages # save multiple matplotlib plots to a PDF
import numpy as np                                   # numerical operations, arrays, and math functions

---
## 3. Upload Files to Google Colab
Since Google Colab is run on Linux-based cloud machines, we need to transfer the files from our PC (Windows, macOS, etc.) to the Google Colab workspace. Prior to running the next code cell, download the two data files from the GitHub code repository: https://github.com/furtman/WQTC25_workshop_IMTR/tree/main/Ex_geospatial/Data

In [ ]:
# Choose files to upload; By default, uploaded files are saved in the "Files/content/" folder within the Google Colab workspace
# from google.colab import files

uploaded_files = files.upload()

---
## 4. Load and Inspect Data
Read in the CSV files to the notebook as variables called `sites` and `wq_data`.

In [ ]:
# Read in stream site location data and inspect
# import pandas as pd

sites = pd.read_csv("streams_sites.csv")
sites # show data

In [ ]:
# Show some general info about the data types of each column in the stream site data
sites.info()

In [ ]:
# Filter data within a specific column, such as SITE_NAME
sites['SITE_NAME']

In [ ]:
# View each stream site name using the unique() function
sites['SITE_NAME'].unique()

In [ ]:
# Filter data for a specific stream site location, such as Deschutes @ Tumwater Falls Park
sites[sites['SITE_NAME'] == 'Deschutes @ Tumwater Falls Park']

## 🛑 Task 1: Read in water quality data and inspect

In [ ]:
# T1.1: Read in water quality data as a variable named "wq_data" and inspect
wq_data = pd.read_csv("wqp_data.csv")
wq_data.info()
wq_data

In [ ]:
# T1.2: Show the unique water quality parameter names
print("The unique parameter names are:", wq_data['parameter'].unique())

In [ ]:
# T1.3: Filter the water quality data for Temperature, water at the Deschutes @ Tumwater Falls Park
wq_data[(wq_data['parameter'] == 'Temperature, water') & (wq_data['SITE_NAME'] == 'Deschutes @ Tumwater Falls Park')]

In [ ]:
# Let's try to visualize the above data by creating a time-series plot of temperature
# import matplotlib.pyplot as plt

# Assign the filtered data to a variable
deschutes_temp_data = wq_data[(wq_data['parameter'] == 'Temperature, water') & (wq_data['SITE_NAME'] == 'Deschutes @ Tumwater Falls Park')]

# Create a time-series plot
plt.plot(deschutes_temp_data['date_time'], deschutes_temp_data['value']) # x, y
plt.show()

# T1.4: What's wrong with the plot?

---
## 5. Data Cleaning and Processing

In [ ]:
# In order to conduct timeseries analysis and plot the WQ data, we need to change the 'date_time' column to a datetime data type
wq_data['date_time_new'] = pd.to_datetime(wq_data['date_time'], format='mixed')
wq_data = wq_data.sort_values(by='date_time_new') # sort values by datetime
wq_data.info()

In [ ]:
# Plot the data with the date_time_new column
deschutes_temp_data = wq_data[(wq_data['parameter'] == 'Temperature, water') & (wq_data['SITE_NAME'] == 'Deschutes @ Tumwater Falls Park')].copy()

# Create a time-series plot
plt.plot(deschutes_temp_data['date_time_new'], deschutes_temp_data['value']) # x, y
plt.show()

---
## 6. Generating Summary Statistics

In [ ]:
# Prior to generating stats, NA or blank values need to be removed from the dataframe so they aren't included in the stats
rows_before = len(wq_data)
print(f"Rows before: {rows_before}")

# Remove NA values from value column
wq_data = wq_data.dropna(subset=['value'])

rows_after = len(wq_data)
print(f"Rows after: {rows_after}")
print(f"Rows removed: {rows_before - rows_after}")

In [ ]:
# Generate summary stats from the deschutes_temp_data variable (Stream temperature @ Deschutes River Tumwater Falls Park)
print(f'Location: {deschutes_temp_data['SITE_NAME'].unique()[0]}')
print(f'Parameter: {deschutes_temp_data['parameter'].unique()[0]} ({deschutes_temp_data['unit'].unique()[0]})')
print(f'Number of observations: {len(deschutes_temp_data)}')
print(f'Min: {deschutes_temp_data['value'].min()}')
print(f'Max: {deschutes_temp_data['value'].max()}')
print(f'Mean: {deschutes_temp_data['value'].mean()}')

In [ ]:
# Convert from C to F and generate summary stats from the temp_data variable (Stream temperature @ Deschutes River Tumwater Falls Park)
deschutes_temp_data['value_F'] = deschutes_temp_data['value'] * 9/5 + 32

print(f'Location: {deschutes_temp_data['SITE_NAME'].unique()[0]}')
print(f'Parameter: {deschutes_temp_data['parameter'].unique()[0]} (\u00B0F)')
print(f'Number of observations: {len(deschutes_temp_data)}')
print(f'Min: {round(deschutes_temp_data['value_F'].min())}')
print(f'Max: {round(deschutes_temp_data['value_F'].max())}')
print(f'Mean: {round(deschutes_temp_data['value_F'].mean())}')

In [ ]:
# Generate some basic summary stats of observed WQ values for each site and parameter pairing
summary_stats = wq_data.groupby(['SITE_NAME', 'parameter'])['value'].agg(['count','mean','median','min','max']).reset_index()
summary_stats

In [ ]:
# View summary stats for a specific site
summary_stats[summary_stats['SITE_NAME'] == 'Deschutes @ Tumwater Falls Park']

## 🛑 Task 2: Identify outliers in wq_data at Deschutes @ Tumwater Falls Park

In [ ]:
# T2.1: Filter wq_data for Deschutes @ Tumwater Falls Park
deschutes_tmf_data = wq_data[wq_data['SITE_NAME'] == 'Deschutes @ Tumwater Falls Park'].copy()

In [ ]:
# T2.2: Create a variable that contains all the unique parameter names in filtered deschutes data
parameters = deschutes_tmf_data['parameter'].unique()

In [ ]:
# Use a for loop to calculate quartiles and IQR for each parameter
for param in parameters:
  print(f"\n{'='*60}")
  print(f"Parameter: {param}")
  param_data = deschutes_tmf_data[deschutes_tmf_data['parameter'] == param].copy() # T2.3: XXX - Filter data for a parameter

  # Calculate quartiles and IQR
  Q1 = param_data['value'].quantile(0.25) #T2.4: XXX - Calculate quartiles for parameter values
  Q3 = param_data['value'].quantile(0.75) #T2.5: XXX - Calculate quartiles for parameter values
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  # Identify outliers
  outliers = param_data[
      (param_data['value'] < lower_bound) | (param_data['value'] > upper_bound) #T2.6: XXX - Identify parameter value outliers
  ]

  # Print results
  print(f"Q1 = {Q1:.2f}, Q3 = {Q3:.2f}, IQR = {IQR:.2f}")
  print(f"Lower bound = {lower_bound:.2f}, Upper bound = {upper_bound:.2f}")
  print(f"Number of outliers = {len(outliers)}")


---

## 7. Generating Plots

### Statisical Summary Plots
In addition to calculating quartiles and identifying outliers, we can also plot the data's distribution.

In [ ]:
# Basic boxplot
# import matplotlib.pyplot as plt

# Filter pH data
deschutes_pH_data = deschutes_tmf_data[deschutes_tmf_data['parameter'] == 'pH'][['value']]

plt.boxplot(deschutes_pH_data['value'])
plt.show()

In [ ]:
# Basic histogram
plt.hist(deschutes_pH_data['value'])
plt.show()

### Time-Series Plots

In [ ]:
# Basic line plot
plt.plot(deschutes_temp_data['date_time_new'], deschutes_temp_data['value_F']) # x, y
plt.show()

In [ ]:
# Basic line plot but a little fancy
plt.figure(figsize=(14, 6)) # width, height
plt.plot(deschutes_temp_data['date_time_new'], deschutes_temp_data['value_F'], # x, y
         marker='o', markersize=3, # marker style
         linestyle='-', linewidth=1) # line style
plt.xlabel('Date') # x-axis label
plt.ylabel('Temperature, water (\u00B0F)') # y-axis label
plt.title('Deschutes @ Tumwater Falls Park') # plot title
plt.grid(True, linestyle='--', alpha=0.8) # background grid
plt.show()

In [ ]:
# Even fancier plot with summary stats and trend line
# from scipy.stats import linregress

# Location and parameter name
site_name = deschutes_temp_data['SITE_NAME'].unique()[0]
param_name = deschutes_temp_data['parameter'].unique()[0]

# Calculate stats
mean_temp = deschutes_temp_data['value_F'].mean()
min_temp = deschutes_temp_data['value_F'].min()
max_temp = deschutes_temp_data['value_F'].max()

# Linear regression for trend
# x as ordinal dates
x = deschutes_temp_data['date_time_new'].map(lambda dt: dt.toordinal())
y = deschutes_temp_data['value_F']
slope, intercept, r_value, p_value, std_err = linregress(x, y)

# Calculate trend line y values
slope_per_year = slope * 365.25 # convert to per year
trend_line = slope * x + intercept

# Compute R² and p-value significance
r_squared = r_value ** 2
if p_value < 0.05:
  significance = "Significant (p < 0.05)"
else:
  significance = "Not Significant (p > 0.05)"
#significance = "Significant" if p_value < 0.05 else "Not Significant" # how the cool kids do it

# List of stats that will be added to the plot
stats_text = (
    f"Max: {max_temp:.1f}°F\n"
    f"Min: {min_temp:.1f}°F\n"
    f"Mean: {mean_temp:.1f}°F\n"
    f"Trend slope: {slope_per_year:.3f}°F/year\n"
    f"R²: {r_squared:.3f}\n"
    f"p-value: {p_value:.4f}\n"
    f"{significance}"
)

# Plotting
plt.figure(figsize=(14, 6)) # width, height
plt.plot(deschutes_temp_data['date_time_new'], deschutes_temp_data['value_F'], # x, y
         marker='o', markersize=3, # marker style
         linestyle='-', linewidth=1, # line style
         label='Observed Data') # legend label
plt.plot(deschutes_temp_data['date_time_new'], trend_line, # trend line
         linestyle='--', linewidth=2, color='red', # line style
         label='Trend Line') # legend label

# Add stats to plot
plt.text(0.01, 0.98, stats_text, transform=plt.gca().transAxes,
         verticalalignment='top', fontsize=10, bbox=dict(facecolor='white', alpha=0.5, edgecolor='gray'))

# Plot beautification
plt.grid(True, linestyle='--', alpha=0.8) # background grid
plt.title(f'{param_name} at {site_name}', fontsize=14, fontweight='bold') # plot title
plt.ylabel("Water Temperature (°F)", fontsize=12) # y-axis label
plt.xticks(rotation=45) # rotate the x-axis labels
plt.legend() # add legend
plt.tight_layout() # auto adjust spacing to prevent overlapping
plt.show()

### Interactive Plots

In [ ]:
# Generate a basic interactive plot of water temperature at every site on the Deschutes River
# import plotly.express as px

# Filter water temperature data and site names that begin with "Deschutes"
filtered_data = wq_data[
    (wq_data['parameter'] == param_name) & # 'Temperature, water'
    (wq_data['SITE_NAME'].str.startswith('Deschutes'))
    ].copy()

# Plotting
fig = px.line(
    filtered_data,
    x='date_time_new', # x data
    y='value', # y data
    color='SITE_NAME', # symbolize site name by color
    title='Deschutes River Water Temperature'
)

# Layout styling for comparison
fig.update_layout(hovermode='x unified') # enables value hovering for easy comparison

fig.show()

In [ ]:
# Generate a fancy interactive plot of all WQ parameters at every site on the Deschutes River

# Filter site names that begin with "Deschutes" and sort by datetime
deschutes_data = wq_data[
    (wq_data['SITE_NAME'].str.startswith('Deschutes'))
    ]

# Plotting
fig_deschutes = px.line(
    deschutes_data,
    x='date_time_new',
    y='value',
    color='SITE_NAME',
    facet_col='parameter', # each parameter gets its own plot
    facet_col_wrap=3, # 3 plots per row
    title='Deschutes River Water Quality Parameters',
    height=800, # figure height
    labels={ # customize label names for data hover
        'date_time_new': 'Date',
        'value': 'Measured Value',
        'SITE_NAME': 'Site Name',
        'unit': 'Unit'
    },
    hover_data={ # customize label values for data hover
        'date_time_new': '|%m-%d-%Y', # reformat the datetime to mm/dd/yyyy
        'value': ':.2f', # value with 2 decimals
        'SITE_NAME': True, # inlcude site name in hover
        'unit': True,
        'parameter': False # do not include parameter name in hover
    }
)

# Improve layout styling
fig_deschutes.update_layout(
    hovermode='x unified',
    showlegend=True,
    legend_title_text='Site Name', # update the legend title
    title_font_size=18, # adjust plot title font size
    margin=dict(l=40, r=40, t=80, b=40) # adjust plot margins (left, right, top, bottom)
)

# Make each facet plot's y-axis values independent from each other (needed since parameters have different measurement units)
fig_deschutes.update_yaxes(title='', matches=None) # don't show y-axis label
fig_deschutes.update_xaxes(title='') # don't show x-axis label

# Clean up facet titles: remove "parameter_unit=" prefix
for annotation in fig_deschutes.layout.annotations:
    annotation.text = annotation.text.replace("parameter=", "")

fig_deschutes.show()

---
## 8. Automating Plot Generation and PDF Reports
Up until this point, Excel would have been able to handle everything we just did (i.e., look at the data, filter by parameter and site, create plots). With Python (and programming in general), we can take this a step further by automating those redundant processes that are done in Excel. For example, we can write a script to plot a timeseries of all WQ parameters at each stream site location and then save to a PDF.

## 🛑 Task 3: Fill in the blanks (XXX) to generate a PDF report of WQ parameter time-series plots at each stream site location

In [ ]:
# Generate a fancy, multi-facet plot of all WQ parameters for each stream site location and save to a single PDF (takes approx. 4-5 minutes to run)
# from matplotlib.backends.backend_pdf import PdfPages
# import numpy as np

# Toggle switch for running the cell
run_cell = True # True or False

# Output file
output_pdf = "WaterQuality_SiteParameter_Trends.pdf"

if run_cell:
  # Create PDF file to save all plots; each site gets a single PDF page
  with PdfPages(output_pdf) as pdf:
      # Loop through each site
      for site_name, site_df in wq_data.groupby('SITE_NAME'):
          parameters = site_df['parameter'].unique()

          # Setup multi-facet plot layout
          n_params = len(parameters)
          ncols = 2
          nrows = int(np.ceil(n_params / ncols))
          fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, nrows * 4))
          axes = axes.flatten()

          # Loop through each parameter, apply regression, calculate stats, and plot
          for i, param_name in enumerate(parameters):
              ax = axes[i]
              param_df = site_df[site_df['parameter'] == param_name].copy()
              param_df = param_df.sort_values(by="date_time_new")

              # Convert temperature from C to F
              if "temp" in param_name.lower():
                  param_df['value_plot'] = param_df['value'] * 9/5 + 32
                  y_label = "Temperature (°F)"
              else:
                  param_df['value_plot'] = param_df['value']
                  y_label = f"{param_name} ({param_df['unit'].iloc[0] if 'unit' in param_df.columns else ''})"

              # Data for regression
              param_df = param_df.dropna(subset=['date_time_new', 'value_plot']) # remove NA values for regression analysis
              x = param_df['date_time_new'].map(lambda dt: dt.toordinal())
              y = param_df['value_plot']
              n_points = len(y)

              # Check data sufficiency
              if n_points >= 3:
                  slope, intercept, r_value, p_value, std_err = linregress(x, y)
                  slope_per_year = slope * 365.25
                  r_squared = r_value ** 2
                  trend_line = slope * x + intercept
                  significance = "Significant (p < 0.05)" if p_value < 0.05 else "Not Significant"
              else:
                  slope_per_year = np.nan
                  r_squared = np.nan
                  p_value = np.nan
                  trend_line = [np.nan] * len(x)
                  significance = "Insufficient data for regression"

              # Summary statistics
              mean_val = y.mean()
              min_val = y.min()
              max_val = y.max()

              # Stats text
              if np.isnan(p_value):
                  p_str = "N/A"
              else:
                  p_str = f"{p_value:.4f}"

              stats_text = (
                  f"n = {n_points}\n"
                  f"Max: {max_val:.1f}\n"
                  f"Min: {min_val:.1f}\n"
                  f"Mean: {mean_val:.1f}\n"
                  f"Trend: {slope_per_year:.3f}/yr\n"
                  f"R²: {r_squared:.3f}\n"
                  f"p: {p_str}\n"
                  f"{significance}"
              )

              # Plotting
              ax.plot(param_df['date_time_new'], y, marker='o', markersize=3,
                      linestyle='-', linewidth=1, label='Observed')
              ax.plot(param_df['date_time_new'], trend_line, linestyle='--',
                      linewidth=2, color='red', label='Trend')
              ax.text(0.01, 0.98, stats_text, transform=ax.transAxes, fontsize=9,
                      verticalalignment='top',
                      bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray'))
              ax.set_title(f"{param_name}", fontsize=11, fontweight='bold')
              ax.set_ylabel(y_label)
              ax.grid(True, linestyle='--', alpha=0.6)
              ax.legend(fontsize=8)
              ax.tick_params(axis='x', rotation=45)

          # Remove unused subplots
          for j in range(i + 1, len(axes)):
              fig.delaxes(axes[j])

          # Layout and save
          fig.suptitle(f"Water Quality Trends at {site_name}", fontsize=14, fontweight='bold')
          plt.tight_layout(rect=[0, 0, 1, 0.96])
          pdf.savefig(fig)
          plt.close(fig)

  print(f"All plots saved to: {output_pdf}")

else:
  print("Skipping this cell.")

---
## 9. Generating Maps
Since the stream site location data has a latitude and longitude column, we can plot the locations on an interactive map.

In [ ]:
# Generate a basic location map of stream site locations

site_map = px.scatter_mapbox(
    sites, # stream site location data
    lat="LAT", # latitude column
    lon="LON", # longitude column
    hover_name="SITE_NAME", # popup label that appears when hovered over
    hover_data="AquaticLifeUse", # popup data that appears when hovered over
    title="Stream Site Monitoring Locations", # map title
    mapbox_style="open-street-map", # basemap; other options - "carto-positron", "carto-darkmatter"
)

site_map.show(config={'scrollZoom': True}) # show map (and enable scroll zoom for convience)

In [ ]:
# In order to conduct geospatial analysis and generate maps of the wq_data, we need to join/merge the WQ data with the stream site location data
merged_data = pd.merge(
    wq_data,
    sites[['SITE_NAME', 'LAT', 'LON', 'AquaticLifeUse']], # keep only the site name, latitude, longitude, and aquatic life use columns
    on='SITE_NAME', # merge the dataframes based on a common column
    how='left' # keep all rows in the wq_data and only matching rows in the sites data
)

merged_data.columns # show the dataframe column names

In [ ]:
# Generate a map showing the distribution of water temperature at stream site locations

# Filter out stream temperature data
stream_temp_data = merged_data[
    (merged_data['parameter'] == param_name) # 'Temperature, water'
    ].copy()

# Calculate monthly average stream temperature for all sites
stream_temp_data["year_month"] = stream_temp_data["date_time_new"].dt.to_period("M").dt.strftime("%Y-%m") # add a year_month column so we can group by month
monthly_avg_temp = (
    stream_temp_data
    .groupby(["SITE_NAME", "LAT", "LON", "year_month", "parameter", "unit"], as_index=False)
    .agg(mean_temp=("value", "mean"))
).sort_values(by="year_month")

# Find min and max stream temperautre values
vmin = monthly_avg_temp['mean_temp'].min()
vmax = monthly_avg_temp['mean_temp'].max()

# Plotting
stream_temp_map = px.scatter_mapbox(
    monthly_avg_temp,
    lat="LAT",
    lon="LON",
    color="mean_temp", # symbolize points by stream temp value
    range_color=(vmin, vmax), # color range based on min/max temp values
    animation_frame="year_month", # animate points by date/time
    hover_name="SITE_NAME",
    hover_data={
        "year_month": "|%Y-%m", # show datetime in hover
        "parameter": True, # show parameter
        "mean_temp": ":.2f", # show value with 2 decimals
        "unit": True, # show parameter unit
        "LAT": False, # DO NOT show LAT
        "LON": False # DO NOT show LONG
    },
    title="Stream Temperature at Monitoring Locations in Thurston County",
    height=800, # map height
    zoom=9, # set zoom level
    mapbox_style="carto-positron" # better basemap for viewing symbol colors
)

# Update map aesthetics
stream_temp_map.update_traces(marker=dict(size=12)) # increase size of points on map

stream_temp_map.show(config={'scrollZoom': True}) # show map (and enable scroll zoom for convience)

---
## 10. Generating Data Dashboards

In [ ]:
# Install packages that aren't preinstalled in Colab
!pip install dash

from dash import Dash, dcc, html, Input, Output

In [ ]:
#  Generate a data dashboard

# Get unique parameter names and sort alphabetically
param_options = sorted(merged_data["parameter"].dropna().unique())

# Initialize the app
app = Dash(__name__)

# App layout (User-Interface) --------------------------------------------------
app.layout = html.Div([
    # Dropdown menu
    html.Div([
        html.Label("Select Parameter:", style={"fontWeight": "bold"}),
        dcc.Dropdown(
            id="param",
            options=[{"label": p, "value": p} for p in param_options],
            value=param_options[0],
            clearable=False,
            style={"width": "100%"}
        ),
    ], style={"width": "25%", "margin": "10px"}),

    # Time-Series Plot and Map
    html.Div([
        html.Div([
            dcc.Graph(
                id="timeseries",
                style={"width": "100%", "height": "90vh"},
                config={"responsive": True}
            )
        ], style={"flex": "1.5"}),

        html.Div([
            dcc.Graph(
                id="map",
                style={"width": "100%", "height": "90vh"},
                config={"responsive": True}
            )
        ], style={"flex": "1"})
    ], style={
        "display": "flex",
        "flexDirection": "row",
        "width": "100%",
        "alignItems": "stretch",
    })
])

# Callback to update plot and map ----------------------------------------------
@app.callback(
    Output("timeseries", "figure"),
    Output("map", "figure"),
    Input("param", "value")
)

# Update plot and map based on user selection from dropdown menu
def update_dashboard(param):
    df = merged_data[merged_data["parameter"] == param].copy()
    df["year_month_dt"] = df["date_time_new"].dt.to_period("M").dt.to_timestamp()
    df["year_month"] = df["date_time_new"].dt.strftime("%Y-%m")

    # Monthly averages
    monthly_avg = (
        df.groupby(["SITE_NAME", "LAT", "LON", "year_month", "year_month_dt"], as_index=False)
          .agg(mean_value=("value", "mean"))
          .sort_values("year_month_dt")
    )

    ## Time-series line plot ---------------------------------------------------
    ts_fig = px.line(
        monthly_avg, x="year_month", y="mean_value", color="SITE_NAME",
        title=f"{param} — Monthly Trend by Site", height=800,
        category_orders={"SITE_NAME": sorted(monthly_avg["SITE_NAME"].unique())}
    )
    ts_fig.update_layout(
        autosize=True,
        xaxis_title="",
        margin=dict(t=50, l=20, r=20, b=100),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.2,
            xanchor="center",
            x=0.5,
            title=None,
            font=dict(size=10)
        )
    )


    ## Map (animated) ----------------------------------------------------------
    vmin, vmax = monthly_avg["mean_value"].min(), monthly_avg["mean_value"].max() # min and max of parameter data

    # Plotting
    map_fig = px.scatter_mapbox(
        monthly_avg, lat="LAT", lon="LON", color="mean_value",
        range_color=(vmin, vmax), animation_frame="year_month_dt",
        hover_name="SITE_NAME", mapbox_style="carto-positron",
        zoom=8, height=800
    )
    map_fig.update_traces(marker=dict(size=12))
    map_fig.update_layout(
        autosize=True,
        margin=dict(t=50, l=0, r=0, b=0),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=0,
            xanchor="center",
            x=0.5,
            title=None
        )
    )

    return ts_fig, map_fig

# Run app ----------------------------------------------------------------------
app.run(mode="inline") # run app within Colab